# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [59]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [60]:
# your code here

#read the dataset and store as a variable
orders_read_csv = pd.read_csv("Orders.zip") 

#create a dataframe from the variable called orders
orders = pd.DataFrame(orders_read_csv)

#display the dataframe head to overview the data
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [61]:
#aggregates the cutomers by Customer ID using groupby() and amount_spent() method
orders_agg = orders.groupby('CustomerID').sum()

#creates a dataframe with the result
orders_agg_df = pd.DataFrame(orders_agg)

orders_agg_df

,Unnamed: 0,InvoiceNo,year,month,day,hour,Quantity,UnitPrice,amount_spent
CustomerID,,,,,,,,,
12346,61619,541431,2011,1,2,10,74215,1.04,77183.60
12347,42441700,101296926,365971,1383,441,2219,2458,481.21,4310.00
12348,2807120,16869685,62324,257,111,472,2341,178.71,1797.24
12349,35444274,42165457,146803,803,73,657,631,605.10,1757.55
12350,1365627,9231629,34187,34,51,272,197,65.30,334.40
...,...,...,...,...,...,...,...,...,...
18280,1110495,5457120,20110,30,10,90,45,47.65,180.60
18281,1560699,3895248,14077,42,49,70,54,39.36,80.82
18282,4642134,6838540,24132,116,60,146,103,62.39,178.05


In [62]:
#find the 95th percentile using quantile() method

orders_agg_df.amount_spent.quantile(0.95)



5840.181999999983

In [63]:
#find the 75th percentile using quantile() method

orders_agg_df.amount_spent.quantile(0.75)

1661.6400000000003

In [64]:

# create an object with the VIP customers above the 95th percentile using quantile() method
orders_vip = orders_agg_df[orders_agg_df.amount_spent > orders_agg_df.amount_spent.quantile(.95)]

#check it looks right
orders_vip.head()


,Unnamed: 0,InvoiceNo,year,month,day,hour,Quantity,UnitPrice,amount_spent
CustomerID,,,,,,,,,
12346,61619,541431,2011,1,2,10,74215,1.04,77183.60
12357,58262250,75290940,263441,1441,917,2096,2708,438.67,6207.67
12359,59103884,138190725,498728,1508,783,3136,1622,2137.86,6372.58
12409,30112571,61157926,219199,789,499,1222,5551,389.62,11072.67
12415,164201777,398543981,1439876,4254,2169,8061,77670,2097.08,124914.53


In [65]:
# create an object with the preffered customers between the 95th and 75th percentile using quantile() method
orders_preffered = orders_agg_df[(orders_agg_df.amount_spent.quantile(.95) > orders_agg_df.amount_spent) & (orders_agg_df.amount_spent > orders_agg_df.amount_spent.quantile(.75))]
orders_preffered.head()

,Unnamed: 0,InvoiceNo,year,month,day,hour,Quantity,UnitPrice,amount_spent
CustomerID,,,,,,,,,
12347,42441700,101296926,365971,1383,441,2219,2458,481.21,4310.00
12348,2807120,16869685,62324,257,111,472,2341,178.71,1797.24
12349,35444274,42165457,146803,803,73,657,631,605.10,1757.55
12352,22083029,47523155,170935,552,243,1193,536,1354.11,2506.04
12356,6327889,32183405,118649,142,185,592,1591,188.87,2811.43


In [67]:
#not used anymore
orders_agg_df['VIP'] = np.where(((orders_agg_df.amount_spent > np.percentile(orders_agg_df.amount_spent, 0.95))  (orders_agg_df.amount_spent < np.percentile(orders_agg_df.amount_spent, 0.75 > 0.95))), 'VIP', 'Non-VIP')

orders_agg_df.head()

,Unnamed: 0,InvoiceNo,year,month,day,hour,Quantity,UnitPrice,amount_spent,VIP
CustomerID,,,,,,,,,,
12346,61619,541431,2011,1,2,10,74215,1.04,77183.60,VIP
12347,42441700,101296926,365971,1383,441,2219,2458,481.21,4310.00,VIP
12348,2807120,16869685,62324,257,111,472,2341,178.71,1797.24,VIP
12349,35444274,42165457,146803,803,73,657,631,605.10,1757.55,VIP
12350,1365627,9231629,34187,34,51,272,197,65.30,334.40,VIP


In [12]:
#not used anymore
orders.quantile(q = 0.05, axis = 0, numeric_only = True, interpolation = 'linear') 

Unnamed: 0       30399.15
InvoiceNo       538863.00
year              2010.00
month                1.00
day                  1.00
hour                 9.00
Quantity             1.00
UnitPrice            0.42
CustomerID       12627.00
amount_spent         1.25
Name: 0.05, dtype: float64

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [ ]:
# your code here

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [ ]:
# your code here